build_dataset 해당

In [ ]:
from core.macro_classes.macro_class_dataset import MacroAData

macro_agent = MacroAData()
macro_agent.fetch_data()
macro_agent.add_features()
macro_agent.save_csv()
macro_agent.make_close_price()

In [ ]:
from core.macro_classes.macro_class_dataset import MacroAData
# macro_agent = MacroAData()
macro_agent.model_maker()

$$$$   X = macro_sercher(agent, ticker) 역할

In [ ]:
from agents.macro_agent import MacroPredictor
from datetime import datetime

macro_sub = MacroPredictor(
    agent_id='MacroSentiAgent',
    base_date=datetime.today(),
    window=40,
    ticker="NVDA"
)
macro_sub.load_assets()               # 모델, 스케일러 등 불러오기


In [ ]:
macro_sub.fetch_macro_data()          # macro_df 불러오기
X_seq, X_scaled = macro_sub.prepare_features()  # 입력 시퀀스 준비

In [ ]:
print(f"X_seq: {X_seq}")
print(f"X_scaled: {X_scaled}")

$$$$   macro_predictor(X) 역할

In [ ]:
pred_prices, target = macro_sub.m_predictor(X_seq)

In [ ]:
print(f"pred_prices: {pred_prices}")
print(f"target: {target}")

llm 생성 용.. macro_reviewer_draft

In [ ]:
 total_json, opinion = macro_sub.macro_reviewer_draft(X_scaled, pred_prices, target)
 

In [ ]:
import numpy as np

# print("원본 X 평균:", np.mean(X, axis=0)[:10])
print("스케일링 후 X 평균:", np.mean(X_scaled, axis=0)[:10])
print("스케일링 후 X 표준편차:", np.std(X_scaled, axis=0)[:10])
print("pred_prices:", pred_prices)


In [ ]:
print(macro_sub.stockdata.MacroSentiAgent["feature_importance"])


In [ ]:
print(f"total_json: {total_json}")

In [ ]:
print(f"opinion: {opinion}")

reviewer_rebut

In [ ]:
# reviewer_rebut()

[토론 테스트]

In [1]:
from agents.debate_agent import DebateAgent

debate_agent = DebateAgent(1, "NVDA")

In [ ]:
debate_agent.run_dataset()

In [ ]:
for agent_id, agent in debate_agent.agents.items():
    scaler = getattr(agent, "scaler", None)
    y_scaler = getattr(scaler, "y_scaler", None)
    print(f"{agent_id}:", type(y_scaler), y_scaler)
    print(agent.ticker)
    # if agent_id =='MacroSentiAgent':
    #     print(agent.macro_df.columns[:20])


In [2]:
opinions = debate_agent.get_opinion(0, "NVDA")

MacroSentiAgent의 데이터 로드.. macro_sercher
[INFO] 모델 및 스케일러 로드 중...
model_path: models/NVDA_MacroSentiAgent.keras
[OK] 모델 및 스케일러 로드 완료
[INFO] MacroSentimentAgent 데이터 수집 중...
1️⃣ Collecting macro features (15 tickers)...


[*********************100%***********************]  15 of 15 completed


[MacroSentimentAgent] Macro data: (43, 91)
   ↳ Downloading NVDA ...


[*********************100%***********************]  1 of 1 completed


[MacroSentimentAgent] Stock data: (42, 9)
[MacroSentimentAgent] Data shape: (42, 99)
[MacroSentimentAgent] Feature engineering complete. Final shape: (42, 170)
[OK] 매크로 데이터 수집 완료: (42, 171)
[INFO] 피처 정리 및 스케일링 중...
[Check] 입력 피처 수: 169 / 스케일러 기준 피처 수: 169
[OK] 스케일링 및 시퀀스 변환 완료
■ macro_sercher StockData 생성 완료 (NVDA)
MacroSentiAgent의 예측
[INFO] 예측 수행 중...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
NVDA: 마지막 종가=198.20 → 예측 종가=201.62 (예상 수익률 1.73%)

================= 예측 결과 (표) =================
  Ticker  Last_Close  Predicted_Close  Predicted_Return  Predicted_%  \
0   NVDA  198.195001       201.622755          0.017295     1.729486   

   uncertainty  confidence  
0     0.050365   19.855179  

================= 예측 결과 (값) =================
{'NVDA': 201.62275518478705}
[MacroSentiAgent] LLM (macro_reviewer_draft)
1️⃣ Collecting macro features (15 tickers)...


[*********************100%***********************]  15 of 15 completed


[MacroSentimentAgent] Macro data: (43, 91)
   ↳ Downloading NVDA ...


[*********************100%***********************]  1 of 1 completed


[MacroSentimentAgent] Stock data: (42, 9)
[MacroSentimentAgent] Data shape: (42, 99)
[MacroSentimentAgent] Feature engineering complete. Final shape: (42, 170)

3️⃣ Running GradientAnalyzer for interpretability...
[INFO] Running Gradient × Input + Integrated Gradients analysis...
[DEBUG] Gradient × Input output shape: (1, 42, 169)
[DEBUG] x_input shape before IG: (1, 42, 169)
[DEBUG] interpolated shape: (51, 42, 169)
[INFO] IG–G×I correlation (agreement_ratio): 0.8402
[INFO] Gradient analysis completed successfully.

4️⃣ Generating explanation using LLM...

================= pred_prices:{'NVDA': 201.62275518478705} =================

================= LLM Explanation =================
(1) Feature Trend (Temporal) 분석: 최근 시점에서 'Open_GC=F'와 'High_GC=F'의 영향력이 증가하였으며, 특히 금 선물 가격(Open_GC=F)의 중요도가 가장 높아졌다. 반면, 다른 전통적 주가지수 변수들의 상대적 중요도는 다소 낮아졌다. 이는 최근 금 가격 변동성이 커지고, 금이 안전자산으로서 투자자들의 관심을 받는 거시적 환경 변화와 연관된다. 이러한 변수 영향력 증가는 모델이 금 가격 변동을 통해 시장 불확실성 및 위험 회피 심리를 반영하고 있음을 시사한다. (2) Model Consistency 

In [ ]:
print(f"opinions:{opinions}")

In [ ]:
print(f"opinions:{opinions.get('MacroSentiAgent')}")

리뷰탈 해보기..

In [3]:
debate_agent.get_rebuttal(1)

 Round 1 rebuttals 생성 완료 (2 agents)


[Rebuttal(from_agent_id='MacroSentiAgent', to_agent_id='SentimentalAgent', stance='SUPPORT', message='현재 거시경제 국면은 인플레이션 압력 완화와 유동성 공급 지속에 따른 확장 국면으로 판단됩니다. 주요 변수 중 Open_GC=F(금 시가, 중요도 0.0127)와 High_GC=F(금 고가, 0.0080), Adj Close_^IXIC(나스닥 종가, 0.0077)가 높은 중요도를 보이며, IG와 G×I 간 일관성도 높아(일치도 0.84) 모델 신뢰도가 견고합니다. 금 가격 상승은 안전자산 선호와 인플레이션 헤지 수요를 반영하며, 나스닥 지수 상승은 기술주 중심의 투자심리 개선을 시사합니다. 민감도 분석에서 Open_GC=F와 Volume_^IXIC가 단기 변동성 요인이나, Adj Close_^IXIC와 Open_GC=F는 안정적 중요도 분포로 구조적 상승 모멘텀을 뒷받침합니다. 상대 예측(next_close 87.20)은 감성 지표 기반 부정적 전망이나, 거시경제 변수와 기술적 지표의 긍정적 신호와는 방향성이 상이해 현실적 타당성이 낮습니다. 따라서 유동성 축소 없이 금리 안정과 달러 강세 제한적 상황에서 금 가격과 나스닥 상승세가 주가 상승을 견인하는 흐름과 부합하는 우리의 예측(201.62)이 거시경제 흐름과 더 일치합니다.'),
 Rebuttal(from_agent_id='SentimentalAgent', to_agent_id='MacroSentiAgent', stance='REBUT', message='두 에이전트의 예측 차이가 매우 크며, 이는 시장 심리와 여론 해석에 있어 일관성이 부족함을 시사합니다. SentimentalAgent는 부정적 감성 지표와 뉴스 흐름을 근거로 약 53.7% 하락을 예측하는 반면, MacroSentiAgent는 기술적 지표와 거시경제 안정성을 바탕으로 강한 상승 모멘텀을 주장합니다. 그러나 MacroSentiAgent의 신뢰도가 19.85로 

get_revise 해보기

In [ ]:
debate_agent.get_revise(1)

'''
에러 나오는 중..
[TechnicalAgent] fine-tuning 실패: 'NoneType' object has no attribute 'parameters'
[SentimentalAgent] fine-tuning 실패: 'NoneType' object has no attribute 'parameters'
'''

결과 도출해보기

In [ ]:
debate_agent.get_ensemble()

통합해서 한번에 실행하기: 기본 라운드는 총 3회. 클래스 정의떄 수정 가능

In [ ]:
#debate_agent.run()